In [8]:
#Package importation
import numpy as np #Numerical tools package
import matplotlib.pyplot as plt #Plotting package
from scipy.optimize import curve_fit #Curve fitting package
import statsmodels.api as sm #Autocorrelation function package
import pandas as pd #.csv and .xlsx management package

<h1>Essential functions for the code to work</h1>

In [9]:
def diffusionModel(t,A,td,c): 
    """Takes a time array, an A (=1/N) value, a diffusion time (t_d) value and a displacement constant.
    Returns an array corresponding to the 2D diffusion model."""
    return A/(1+(t/td))+c

def fitCurve(time,ac):
    """Takes a time array and an experimental autocorrelation array. 
    Returns the fitted curve, the fit A (1/N) and td values respectively and the covariance matrix of the fit"""
    pOpt,pCov = curve_fit(diffusionModel,time,ac) #Generates the optimal parameters (pOpt) and the covariance matrix (pCov)
    fitCurve = diffusionModel(time,*pOpt) #Optimal curve
    return fitCurve,pOpt[0],pOpt[1],pCov

def rawData(imRoute):
    """Takes an image route corresponding to a fluorescence signal (in .tif format). 
    Returns raw data along with its time vector."""
    dt = 1.12 #ms per measurement (microscope-specific value)
    raw = plt.imread(imRoute)[:,0] #Fluorescence signal (just one pixel of two available)
    tf = len(raw)*1.12 #Final sampling sime
    time = np.arange(0,tf+dt,dt) #Time in ms

    return raw,time

def splice(imRoute):
    """Takes an image route corresponding to a 33s-long fluorescence signal (in .tif format). 
    Returns splices of 1 second from such data along with its corresponding time vector"""
    dt = 1.12 #ms per measurement (microscope-specific value)
    n1Sec = int(1000/dt) #Data points that make up to a second
    raw, time = rawData(imRoute)
    splices = [] #Vector containing all splices extracted from raw
    totalSecs = int(len(raw)/n1Sec) #Total iterations to splice all data
    time = np.arange(0,n1Sec*dt,dt) #Time vector for each splice

    for i in range(totalSecs):
        prStep = i*n1Sec #starting splicing step
        nextStep = (i+1)*n1Sec #following splicing step
        spl = raw[prStep:nextStep] #current splice
        splices.append(spl)

    return splices,time

def autocorrelation(raw,time):
    """Takes the raw fluorescence signal and its time vector. 
    Gives the associated autocorrelation function with its time vector"""
    corr = sm.tsa.acf(raw,nlags=len(raw),fft = False)[1::] #autocorrelation function (drops 1st value by necessity)
    time = time[0:len(corr)]

    return corr,time

def allAutocorrelations(imRoute):
    """Takes the route of a fluorescence signal (in .tif format). 
    Gives the autocorrelation function of all splices in an
    array."""

    splices, time = splice(imRoute) #Takes all splices of one dataset
    autocorrs = [] #Autocorrelation functions for all splices
    for i in range(len(splices)):
        res = autocorrelation(splices[i],time)
        autocorrs.append(res[0])
        time = res[1]

    return autocorrs, time

<h1>Workflow</h1>
Note: Wherever you see (MANUAL) as a comment, please insert the file name corresponding to your data.
<h3>1. Producing a .csv file with all the autocorrelation functions </h3>

In [10]:
labels = [] #.tif file labels that will be used to perform FCS analysis
ex = [] #.tif file tuples ((i,j) for GUVi FCSj.tif) that DO NOT follow selection criteria

for i in range(1,8): #GUV Iterator
    for j in range(1,8): #Signal iterator
        if (i,j) not in ex: #won't read any signal in the ex list
            labels.append("Pruebas 22-05-24/15 CHO/GUV{} FCS{}.tif".format(i,j)) #(MANUAL)
            #Note: The file route should contain all fluorescence signals that want to be analyzed
       
autocorrs = [] #All autocorrelations obtained from all the files in the labels list

for label in labels:
    actualAC,time = allAutocorrelations(label) #Takes all autocorrelations from a single label
    for ac in actualAC:
       autocorrs.append(ac) #Appends every autocorrelation function to autocorrs

data = {} #Data dictionary (soon to be .csv)
data["Time(ms)"] = time #Time vector is inserted into the data dict for good measure
for i in range(len(autocorrs)): #Inserting all data in the autocorrs list
    data["ACF{}".format(i)] = autocorrs[i]

dataframe = pd.DataFrame(data) #Converting data into a dataframe

#.csv file with all the autocorrelation funtions! Please name it and store it as you prefer
dataframe.to_csv("Pruebas 22-05-24/15 CHO/test.csv",index = False) #(MANUAL)

<h3>2. Fitting all the autocorrelation functions and obtaining diffusion coefficients</h3>

In [11]:
data = pd.read_csv("Pruebas 22-05-24/15 CHO/test.csv") #(MANUAL) Data from the .csv generated above
time = np.array(data["Time(ms)"])
totalACFs = len(data.columns)-1 #Number of ACFs (ignores the time column)
nACFs = 33 #Number of ACFs per measurement 

#Every 33 autocorrelation functions (one total measurement) are averaged and kept in the meanACFs list
meanACFs = [] #Mean autocorrelation functions

for i in range(0,totalACFs,nACFs):
    indices = np.arange(i,i+nACFs) 
    currentACFs = [] #Current analyzed ACFs coming from the indices

    for ind in indices:
        currentACFs.append(np.array(data["ACF{}".format(ind)])) 

    currentACFs = np.array(currentACFs).T #Array containing all 33 ACFs soon to be averaged

    currentMeanACF = [] #Array containing all averaged ACFs
    for j in range(len(currentACFs)):
        currentMeanACF.append(np.mean(currentACFs[j]))

    currentMeanACF = np.array(currentMeanACF)
    meanACFs.append(currentMeanACF/currentMeanACF[0]) #Optional: ACFs are normalized to appreciate the td difference

tds = [] #List containing all the measured diffusion times
r0 = 0.8 #(MANUAL) R0 value (in um) previously calibrated

for meanACF in meanACFs:
    fit,_,td,_ = fitCurve(time,meanACF) #Every curve gets fitted and plotted 
    #plt.semilogx(time,fit/fit[0]) 
    tds.append(td)

tds = np.array(tds)/1000 #Conversion from ms to s
ds = r0**2/(4*tds) #Diffusion coefficient measurement from FCS theory formula

<h3>Notes:</h3>

1. In order to further clean up the diffusion coefficient data, you can use tests such as Chauvenet's principle or the Modified Thomson $\tau$ Test to eliminate possible outliers.
2. You can still use this code if you want to calibrate $R_0$. However, you will need to use the formula $R_0 = \sqrt{4D\tau_D}$ using a fluorophore with a well-known D and your measured $\tau_D$ values.
3. Feel free to make any necesary adaptations, but please cite us if you do so. <b>Happy coding!</b>